## Expanding the Medicinal Chemist Toolbox: Comparing Seven C(sp2)-C(sp3) Cross-Coupling Methods by Library Synthesis

DOI: 10.1021/acsmedchemlett.0c00093

Amanda W. Dombrowski, Nathan J. Gesmundo, Ana L. Aguirre, Katerina A. Sarris, Jonathon M. Young, Andrew R. Bogdan, M. Cynthia Martin, Shasline Gedeon, and Ying Wang *ACS Med. Chem. Lett.* **2020**, *11*, 4, 597-604.

Import schema and helper functions

In [1]:
try:
    import ord_schema
    import rdkit
except:
    import sys
    !wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
    !time bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
    !time conda install -q -y -c rdkit rdkit
    !time conda install -q -y -c anaconda protobuf
    !git clone https://github.com/Open-Reaction-Database/ord-schema.git
    %cd ord-schema
    !python setup.py install
    sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [2]:
import ord_schema
from datetime import datetime
from ord_schema.proto import dataset_pb2
from ord_schema.proto import reaction_pb2
from ord_schema.units import UnitResolver
from ord_schema import validations
from ord_schema import message_helpers

unit_resolver = UnitResolver()

In [4]:
from tqdm import tqdm

In [5]:
import pandas as pd
from rdkit import Chem

## Load the dataset
The original dataset published by the authors is found in ml0c00093_si_001.xlsx, and the "All Data" page was converted to .csv in original_data.csv. This file is missing SMILES strings for many reaction components/products. These were manually filled in by the record submitter: Michael Maser (mmaser@caltech.edu).

In [12]:
# Create pandas dataframe
data = pd.read_csv('completed_data.csv')

# View dataframe sample
print(data.shape)
data.head()

(796, 14)


,Transformation,Core SMILES,Monomer SMILES,Product SMILES,Alkyl Group SMILES [Core],Alkyl Group SMILES [X],Salt,Salt Ratio,Yield (%),Yield,Purified,C(sp3)-X Substitution,Main C(sp3) substituent Type,Remote FG
0,BF3K-Ni-photoredox,BrC1=CC(N2N=CC=C2)=CC=C1,C[B-](F)(F)F.[K+],Cc1cccc(c1)n2cccn2,C[Core],[X]C,NaN,NaN,0,0.0000,0.0,0 - Methyl,alkyl,--
1,CEC-Ni-photoredox,BrC1=CC(N2N=CC=C2)=CC=C1,COS(=O)(=O)c1ccc(C)cc1,Cc1cccc(c1)n2cccn2,C[Core],[X]C,NaN,NaN,10,0.1021,2.1,0 - Methyl,alkyl,--
2,COOH-Ni-photoredox,BrC1=CC(N2N=CC=C2)=CC=C1,CC(=O)O,Cc1cccc(c1)n2cccn2,C[Core],[X]C,NaN,NaN,0,0.0000,0.0,0 - Methyl,alkyl,--
3,BF3K-Pd-Suzuki,BrC1=CC(N2N=CC=C2)=CC=C1,C[B-](F)(F)F.[K+],Cc1cccc(c1)n2cccn2,C[Core],[X]C,NaN,NaN,0,0.0000,NaN,0 - Methyl,alkyl,--
4,MIDA-Pd-Suzuki,BrC1=CC(N2N=CC=C2)=CC=C1,CB1OC(=O)CN(C)CC(=O)O1,Cc1cccc(c1)n2cccn2,C[Core],[X]C,NaN,NaN,0,0.0000,0.0,0 - Methyl,alkyl,--


In [13]:
# Remove unnecessary columns
data = data.drop(['Alkyl Group SMILES [Core]', 'Alkyl Group SMILES [X]', 'Yield', 'C(sp3)-X Substitution', 'Main C(sp3) substituent Type', 'Remote FG'], axis=1)

# View new dataframe
print(data.shape)
data.head()

(796, 8)


,Transformation,Core SMILES,Monomer SMILES,Product SMILES,Salt,Salt Ratio,Yield (%),Purified
0,BF3K-Ni-photoredox,BrC1=CC(N2N=CC=C2)=CC=C1,C[B-](F)(F)F.[K+],Cc1cccc(c1)n2cccn2,NaN,NaN,0,0.0
1,CEC-Ni-photoredox,BrC1=CC(N2N=CC=C2)=CC=C1,COS(=O)(=O)c1ccc(C)cc1,Cc1cccc(c1)n2cccn2,NaN,NaN,10,2.1
2,COOH-Ni-photoredox,BrC1=CC(N2N=CC=C2)=CC=C1,CC(=O)O,Cc1cccc(c1)n2cccn2,NaN,NaN,0,0.0
3,BF3K-Pd-Suzuki,BrC1=CC(N2N=CC=C2)=CC=C1,C[B-](F)(F)F.[K+],Cc1cccc(c1)n2cccn2,NaN,NaN,0,NaN
4,MIDA-Pd-Suzuki,BrC1=CC(N2N=CC=C2)=CC=C1,CB1OC(=O)CN(C)CC(=O)O1,Cc1cccc(c1)n2cccn2,NaN,NaN,0,0.0


# Define a single reaction

The report details 7 cross-coupling reaction types, each with their own general procedure(s). 

**General Procedure A – Suzuki Coupling with Potassium Alkyltrifluoroborate Salts (used in Figure 2 and 3)**

The alkyl reagent (1.3 equiv, 0.13 mmol) and CataCXium A Pd G3 (5 mol%, 3.6 mg, 4.9 x 10-3 mmol) were weighed into a 4 mL vial and purged with nitrogen. A solution of the aryl halide (1 equiv, 0.10 mmol) in 0.45 mL of toluene was added. A nitrogen sparged Cs2CO3 solution (3 equiv, 7 M in H2O, 43 μL) was added, and the reaction was stirred at 100 °C for 72 h. Upon completion, the reaction was concentrated, taken up in 1:1 MeOH/DMSO and filtered through Celite. The product was purified by reverse phase HPLC. This procedure was used for all Suzuki couplings with BF3K salts.

**General Procedure B – Suzuki Coupling with MIDA Boronate Reagents (used in Figure 2)**

The alkyl reagent (1.2 equiv, 0.12 mmol) and SPhos Pd G3 (5 mol%, 3.9 mg, 5.3 x 10-3 mmol) were weighed into a vial and purged with nitrogen. The aryl halide (1 equiv, 0.10 mmol) dissolved in dioxane (0.5 M, 0.20 mL) was added, and the solution was stirred at room temperature for 10 minutes. A nitrogen sparged K3PO4 solution (7.5 equiv, 3.0 M in H2O, 0.25 mL) was added, and the reaction was stirred at 60 °C for 6 h. Upon completion, the reaction was concentrated, taken up in 1:1 MeOH/DMSO and filtered through Celite. The product was purified using reverse phase HPLC. This procedure was used for all Suzuki couplings with MIDA boronates.

**General Procedure C − Nickel/Photoredox Decarboxylative Coupling – activated/stabilized carboxylic acids (used in Figure 2, 3 and 4b)**

The alkyl reagent (carboxylic acid, 1.5 equiv, 0.19 mmol, if solid) and Cs2CO3 (1.5 equiv, 0.19 mmol) were weighed into a 4 mL vial equipped with a stir bar. The vial was moved to a nitrogen-atmosphere dry box. There a solution of the aryl halide core (1 equiv, 0.13 mmol) in 0.65 mL DMA was added to the vial followed by the photocatalyst, Ir(dF(CF3)ppy)2(dtbbpy)PF6 (2 mol%, 2.5 μmol), in 0.30 mL DMA and the nickel catalyst, NiCl2(dtbbpy)3 (5 mol%, 6.3 μmol), in 0.30 mL DMA. Liquid alkyl acid building blocks were dispensed to the vessels at this stage. The vial was capped, placed in the HepatoChem photoreactor, and irradiated for 48 h using 450 nm blue LEDs. Upon completion, the reaction was diluted with 1:1 MeOH/DMSO to 2 mL and filtered. The product was purified by reverse phase HPLC.

**General Procedure D − Nickel/Photoredox Decarboxylative Coupling – alpha-alkyl carboxylic acids (used in Figure 2 and 3)**

The alkyl reagent (carboxylic acid, 1.5 equiv, 0.19 mmol) was weighed into a 4 mL vial equipped with a stir bar. The vial was moved to a nitrogen-atmosphere dry box. There a solution of the aryl halide core (1 equiv, 0.13 mmol) in 0.65 mL DMSO was added to the vial followed by the photocatalyst, Ir(dF(CH3)ppy)2(dtbbpy)PF6 (2 mol%, 2.5 μmol) in 0.30 mL DMSO and the nickel catalyst, NiCl2(dtbbpy) (5 mol%, 6.3 μmol) in 0.30 mL DMSO. Finally, 2-tert-butyl-1,1,3,3-tetramethylguanidine (1.5 equiv, 0.19 mmol) was added to the vial. The vial was capped, placed in the HepatoChem photoreactor, and irradiated for 48 h using 450 nm blue LEDs. Upon completion, the reaction was diluted with 1:1 MeOH/DMSO to 2 mL and filtered. The product was purified by reverse phase HPLC.

**General Procedure E − Nickel/Photoredox Potassium Alkyltrifluoroborate Salt Coupling (used in Figure 2 and 3)**

The potassium alkyltrifluoroborate salt reagent (1.3 equiv, 0.16 mmol), was weighed into a 4 mL vial equipped with a stir bar. The vial was moved to a nitrogen-atmosphere dry box. There a stock solution of the aryl halide core (1 equiv, 0.13 mmol), photocatalyst Ir(dF(CF3)ppy)2(bpy)PF6 (2 mol%, 2.5 μmol), and nickel catalyst NiCl2(dtbbpy) (5 mol%, 6.3 μmol) in 2.5 mL of 4:1 dioxane/DMA was added to the reaction vial. Finally, 2,6-lutidine (2.0 equiv, 0.50 mmol) was added to the vial using a pipette. The vial was capped, placed in the HepatoChem photoreactor, and irradiated for 48 h using 450 nm blue LEDs. Upon completion, the reaction was concentrated to remove dioxane, diluted with 1:1 MeOH/DMSO to 2 mL, and filtered. The product was purified by reverse phase HPLC.

**General Procedure F − Nickel/Photoredox Potassium Alkyltrifluoroborate Salt Coupling – tertiary BF3K salt examples (used in Figure 2)**

The potassium alkyltrifluoroborate salt reagent (2 equiv, 0.25 mmol), was weighed into a 4 mL vial equipped with a stir bar. The photocatalyst, Ir(dF(CF3)ppy)2(bpy)PF6 (1 mol%, 1.3 μmol), the nickel catalyst, Ni(TMHD)2 (10 mol%, 13 μmol), ZnBr2 (10 mol%, 13 μmol), and K2HPO4 (2 equiv, 0.25 mmol) were then dispensed into the vial. The vial was moved to a nitrogen-atmosphere dry box. There a stock solution of the aryl halide core (1 equiv, 0.13 mmol) in 1.25 mL DMA was added to the vial. The vial was sealed, removed from the dry box, placed in the HepatoChem photoreactor, and irradiated for 48 h using 450 nm blue LEDs. Upon completion, the reaction diluted with 1:1 MeOH/DMSO to 2 mL then filtered. The product was purified by reverse phase HPLC.

**General Procedure G − Nickel/Photoredox Cross-Electrophile Coupling (used in Figure 2, 3 and 4a)**

The alkyl bromide (1.5 equiv, 0.19 mmol) was weighed into a 4 mL vial equipped with a stir bar. The vial was moved to a nitrogen-atmosphere dry box. There a solution of the aryl halide core (1 equiv, 0.13 mmol), NiCl2(dtbbpy) (0.5 mol%, 0.70 μmol), and Ir(dF(CF3)ppy)2(dtbbpy)PF6 (1.0 mol%, 1.3 μmol) were added in 1.0 mL DME followed by 2,6-lutidine (2 equiv, 0.26 mmol) and tris(trimethylsilyl)silane (1.2 equiv, 0.15 mmol). The vial was sealed with a teflon cap, placed in the HepatoChem photoreactor, and irradiated for 48 h using 450 nm blue LEDs. Upon completion, the reaction was diluted with 1:1 MeOH/DMSO to 2 mL and filtered. The product was purified by reverse phase HPLC. This procedure was used for all nickel/photoredox cross-electrophile couplings.

**General Procedure H − Nickel-Catalyzed Cross-Electrophile Coupling (used in Figure 2, 3 and 4a)**

A 4 mL scintillation vial was charged with a stir bar, NiCl2 glyme (7.0 μmol, 7 mol%), NaI (0.25 mmol, 25 mol%), ligand (1, 10, or 13) (7.0 μmol, 7 mol %), and Zn Flake (0.20 mmol, 2 equiv). The vial was moved to a nitrogen atmosphere dry box, and 100 μL of a stock solution of the alkyl halide in DMA (0.20 mmol, 2 equiv) was added. To this, 300 μL of a stock solution of the aryl bromide core (0.10 mmol, 1 equiv) in DMA was added followed by the addition of trifluoroacetic acid (0.01 mmol, 10 mol%). The vial was capped using a septa cap and placed to heat at 60°C for 18 hours. Upon completion the crude material was diluted with additional DMA to 2mL and filtered. The product was purified using reverse phase HPLC. This procedure was used for all nickel catalyzed cross-electrophile couplings.

**General Procedure J − Negishi Coupling with Commercial Organozincs (used in Figure 2)**

In a 4 mL vial containing a stir bar was added aryl bromide (0.10 mmol, 1.0 equiv.) in THF (0.5 mL). Pd-PEPPSI- IPentCl (0.025 M in THF, 200 μL, 5.0 μmol, 0.05 equiv) was added. The vial was purged with nitrogen and capped. Alkyl zinc (0.20 mmol, 2.0 equiv) was added and the reaction was stirred overnight at room temperature. Upon completion, the reaction was filtered through a syringe filter and the product was purified via reverse phase HPLC.

**General Procedure K − Negishi Coupling with in situ Formed Organozincs (used in Figure 3)**

In a 4 mL vial containing a stir bar was added aryl bromide (0.10 mmol, 1.0 equiv) in dimethyl acetamide (0.3 mL). SPhos Pd G4 (0.025 M in DMA, 200 μL, 5.0 μmol, 0.05 equiv) was added. The vial was purged with nitrogen and capped. Alkyl zinc (0.20 mmol, 2.0 equiv) was added, and the reaction was stirred overnight at 50 °C. Upon completion, the reaction was filtered through a syringe filter, and the product was purified via reverse phase HPLC.

**General Procedure for in situ Organozinc Formation (used with General Procedure K and in Figure 3)**

Organozinc reagents were prepared using the method described by Alcazar and coworkers.4 Substrates were passed through a 12 g packed bed of granular zinc at a flow rate of 0.5 mL/min, collected under nitrogen, titrated and used without further modification.

(Starting material,
Solvent,
Temperature,
Final Titre)
(t-butyl 2-bromoacetate,
0.54 M in THF,
40 °C,
0.28 M)
(t-butyl 3-iodoazetidine-1-carboxylate,
0.56 M in 0.5 M LiCl in THF,
40 °C,
0.20 M)
(t-butyl 3-iodopyrrolodine-1-carboxylate,
0.48 M in 0.5 M LiCl in THF,
60 °C,
0.05 M)
(t-butyl 4-iodopiperidine-1-carboxylate,
0.50 M in 0.5 M LiCl in THF,
60 °C,
0.15 M)
(4-iodotetrahydro-2H-pyran,
0.52 M in 0.5 M LiCl in THF,
60 °C,
0.28 M)

**Define reaction inputs**:
These reaction conditions are not typical batch or flow, but individual droplets. We take the authors' characterization of the system at face value, where they report an approximate 1:100 dilution of a concentrated 5x1 uL reaction slug. This is equivalent to the addition of 495 uL additional solvent after preparation.

- Reactant 1 is 0.0004 mmol in 1 uL solvent
- Reactant 2 is 1 equiv (0.0004 mmol) in 1 uL solvent
- Reagent 1 (base) is 2.5 equiv (0.001 mmol) in 1 uL solvent
- Ligand is 0.125 equiv (5.0e-5 mmol) in 1 uL solvent
- Catalyst is 0.0625 equiv (2.5e-5 mmol) in 1 uL solvent
- Solvent is 9:1 ratio with water, 495 uL total (49.5 uL water, 445.5 uL solvent)